In [1]:
STOCK_NAME = '효성'

In [2]:
import numpy as np
import os
import pickle
import random
import datetime
from tqdm import tqdm
from multiprocessing import Pool
from MeCab import Tagger
import tensorflow as tf
import tensorflow.keras.backend as ktf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, Concatenate, Input, Embedding, Reshape
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

## GPU Setting

In [3]:
# gpu 설정
def get_session():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5,allow_growth=True,visible_device_list='2')
    return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
ktf.set_session(get_session())

## Hyper Parameters

In [4]:
# 하이퍼파라미터
seq_size = 70  # model input shape[0], 입력 데이터의 형태소 최대 갯수
embed_size = 128  # model input shape[1], 각 형태소의 임베딩 벡터 크기
batch_size = 32 # 각 미니배치의 크리
vocab_size = 455001  # word2index_dict의 단어 수
validation_split = 0.1  # 학습 시 train set에서 validation set으로 사용할 데이터 비율
learning_rate = 0.001
dropout_rate = 0.5
kernel_sizes = [3, 4, 5]  # kernel_size list for each CNN layers
n_class = 1  # 분류할 클래스 수 (binary_crossentropy를 쓰므로 1개 클래스가 0 또는 1의 값을 가지는 것으로 2 클래스 분류)
n_epoch = 13
n_patience = 6  # early stop 콜백의 파라미터

# random seed
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

## 경로 설정

In [5]:
# 파일 경로
root_path = '/data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification'
data_path = f'{root_path}/data/sentences/increased_labeled/labeled_{STOCK_NAME}.txt'
vocab_path = f'{root_path}/data/vocabulary/word2index_dict_190117.pkl'
mecab_path = '-d /data/lib/mecab/mecab-ko-dic-default'

# 저장경로용 시간, 파일명 문자열
now_dt = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
model_name = data_path.split('/')[-1].replace('labeled_', '').replace('.txt', '')

# 텐서보드 디렉토리 생성 및 로그 경로
tensorboard_dir = 'tensorboard'
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)
tblog_path = f'{root_path}/{tensorboard_dir}/{model_name}'

# 체크포인트 디렉토리 생성
ckp_dir = f'{root_path}/checkpoint/{model_name}/'
if not os.path.exists(ckp_dir):
    os.makedirs(ckp_dir)
ckp_path = os.path.join(ckp_dir, now_dt + '_weights.{epoch:03d}-{val_acc:.4f}.hdf5')

## 단어사전 로드
word2index_dict.pkl을 로드한다. 거의 대부분의 단어들을 유니크한 숫자로 인코딩하기 위한 딕셔너리이다. (str -> int)

In [6]:
with open(vocab_path, 'rb') as fp:
    word2index_dict = pickle.load(fp)
    
if vocab_size != len(word2index_dict):
    vocab_size = len(word2index_dict)
print(len(word2index_dict))

455001


## 데이터 로드


In [7]:
def load_data(path):
    with open(data_path, 'r') as fp:
        data = [l.strip() for l in fp.readlines() if len(l) > 10 and len(l.split())]
    data_X = [d.rsplit('#', 1)[0] for d in data]
    data_y = [int(d.rsplit('#', 1)[-1]) for d in data]
    return data_X, data_y

In [8]:
data_X, data_y = load_data(data_path)
print('data_X size : ', len(data_X))
print('data_y size : ', len(data_y))
print('data_X[10] : ', data_X[10])
print('data_y[10] : ', data_y[10])

data_X size :  23229
data_y size :  23229
data_X[10] :  전효성은 3일 자신의 인스타그램에 팬들을 향한 짤막한 셀프 영상을 게재했다
data_y[10] :  0


### !!! 클래스별 데이터 수 비교하기



In [9]:
# 레이블별 데이터 비율 비교
class_0_data = [x for x, y in zip(data_X, data_y) if y == 0]
class_1_data = [x for x, y in zip(data_X, data_y) if y == 1]

print(f'class_0_data size : {len(class_0_data)}')
print(f'class_1_data size : {len(class_1_data)}')

class_0_data size : 11179
class_1_data size : 12050


In [10]:
# sample_x_c0 = class_0_data
# sample_x_c1 = random.sample(class_1_data, 40000)
# data_X = sample_x_c0 + sample_x_c1

## Tokenizing

In [11]:
def tokenize(sentence):
    tagger = Tagger(mecab_path)
    raw_tokens = tagger.parse(sentence).splitlines()
    parsed_tokens = []
    for raw_token in raw_tokens:
        word_tag_ruple = raw_token.split('\t')
        if len(word_tag_ruple) != 2:
            continue
        word_stem = word_tag_ruple[0]
        word_tag = word_tag_ruple[1].split(',')[0]
        if not word_stem or not word_tag:
            continue
        if word_tag in {'NNP', 'NNG', 'SL', 'VV', 'VA', 'VX'}:
            parsed_tokens.append(word_stem)
    return parsed_tokens

In [12]:
tokenized_sents = []
with Pool(processes=8) as pool:
    for tokens in tqdm(pool.imap(tokenize, data_X), total=len(data_X), desc='tokenizing'):
        if tokens:
            tokenized_sents.append(tokens)
            
print('tokenized_sents size : ', len(tokenized_sents))

tokenizing: 100%|██████████| 23229/23229 [00:04<00:00, 5424.24it/s]


tokenized_sents size :  23229


## Encoding

In [13]:
def encode(tokens):
    padded_tokens = list(map(lambda x : tokens[x] if x < len(tokens) else '#', range(seq_size)))
    embed_vect = list(map(lambda w : word2index_dict[w] if w in word2index_dict else 0, padded_tokens))
    return embed_vect

In [14]:
encode_X = []
with Pool(processes=8) as pool:
    for encoded_tokens in tqdm(pool.imap(encode, tokenized_sents), total=len(data_X), desc='encoding'):
        encode_X.append(encoded_tokens)
        
print('encode_X size : ', len(encode_X))

encoding: 100%|██████████| 23229/23229 [00:01<00:00, 13828.42it/s]


encode_X size :  23229


## Split Train - Test Dataset

In [15]:
def get_train_test_data(data_x, data_y):
    
    x_c0, x_c1 = [], []
    for x, y in zip(data_x, data_y):
        if y == 0:
            x_c0.append(x)
        if y == 1:
            x_c1.append(x)
    
    increased_c1 = []
    
    if len(x_c0) >= len(x_c1)*2:
        div_ratio = int(len(x_c0) / len(x_c1))
        for i in range(div_ratio):
            if  (i + 1) * len(x_c1) < len(x_c0):
                increased_c1 += x_c1
    else:
        increased_c1 = x_c1
            
    shuff_c0 = random.sample(x_c0, len(x_c0))
    shuff_c1 = random.sample(increased_c1, len(increased_c1))
    
    c0_tsize = int(len(shuff_c0) * validation_split)
    c1_tsize = int(len(shuff_c1) * validation_split)
    
    test_X = shuff_c0[:c0_tsize] + shuff_c1[:c1_tsize]
    train_X = shuff_c0[c0_tsize:] + shuff_c1[c1_tsize:]
    test_y = [0 for _ in range(c0_tsize)] + [1 for _ in range(c1_tsize)]
    train_y = [0 for _ in range(len(shuff_c0) - c0_tsize)] + [1 for _ in range(len(shuff_c1) - c1_tsize)]    
        
    return np.array(train_X), np.array(train_y), np.array(test_X), np.array(test_y)


In [16]:
train_X, train_y, test_X, test_y = get_train_test_data(encode_X, data_y)

print(f'train_X size : {len(train_X)}')
print(f'train_y size : {len(train_y)}')
print(f'test_X size : {len(test_X)}')
print(f'test_y size : {len(test_y)}')

train_X size : 20907
train_y size : 20907
test_X size : 2322
test_y size : 2322


## Model

In [17]:
def get_char_cnn_model(kernel_sizes, seq_size, vocab_size, embed_size, dropout_rate, n_class, learning_rate):
    # input layer
    inputs = Input(shape=(seq_size,), dtype='float32', name='input')
    
    embedded = Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=seq_size, name='embedded')(inputs)
    reshaped = Reshape((seq_size, embed_size, 1), name='reshape')(embedded)

    # multiple CNN layers
    convolution_layers = []
    for idx, kernel_size in enumerate(kernel_sizes):
        conv = Conv2D(filters=256, kernel_size=(kernel_size, embed_size), padding='valid', activation='relu', name=f'conv_{idx}')(reshaped)
        pool = MaxPooling2D(pool_size=(seq_size - kernel_size + 1, 1), strides=(1, 1), padding='valid', name=f'pool_{idx}')(conv)
        dropout = Dropout(rate=dropout_rate, name=f'dropout_{idx}')(pool)
        convolution_layers.append(dropout)
        
    # concatenate all CNN output tensors
    concat_tensor = Concatenate(axis=1, name='concat')(convolution_layers)    
    
    # middle fully-connected layer
    flatten = Flatten(name='flatten')(concat_tensor)
    hidden = Dense(384, activation='relu', name='hidden')(flatten)
    dropout = Dropout(rate=dropout_rate, name='dropout')(hidden)
    
    # output layer
    outputs = Dense(n_class, activation='sigmoid', name='output')(dropout)

    # connect layers to model
    model = Model(inputs=inputs, outputs=outputs)
    
    # compile model
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [18]:
model = get_char_cnn_model(kernel_sizes, seq_size, vocab_size, embed_size, dropout_rate, n_class, learning_rate)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 70)           0                                            
__________________________________________________________________________________________________
embedded (Embedding)            (None, 70, 128)      58240128    input[0][0]                      
__________________________________________________________________________________________________
reshape (Reshape)               (None, 70, 128, 1)   0           embedded[0][0]                   
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 68, 1, 256)   98560       reshape[0][0]                    
__________________________________________________________________________________________________
conv_1 (Co

## Callbacks

In [19]:
early_stopping = EarlyStopping(patience=n_patience)
# tensorboard = TensorBoard(log_dir=tblog_path, batch_size=batch_size)
checkpoint = ModelCheckpoint(ckp_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_weights_only=False)

In [20]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir=tblog_path, **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()
        
tensorboard = TrainValTensorBoard(batch_size=batch_size)

## Training

In [21]:
history = model.fit(
    train_X, 
    train_y, 
    batch_size=batch_size, 
    epochs=n_epoch,
    validation_data=(test_X, test_y),
    callbacks=[early_stopping, tensorboard, checkpoint]
)

/data/venv/lib64/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20907 samples, validate on 2322 samples
Epoch 1/13
20907/20907 [==============================] - 18s 848us/step - loss: 0.1145 - acc: 0.9535 - val_loss: 0.0413 - val_acc: 0.9871

Epoch 00001: val_loss improved from inf to 0.04131, saving model to /data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/효성/20190308152428_weights.001-0.9871.hdf5
Epoch 2/13
20907/20907 [==============================] - 16s 782us/step - loss: 0.0321 - acc: 0.9893 - val_loss: 0.0353 - val_acc: 0.9871

Epoch 00002: val_loss improved from 0.04131 to 0.03529, saving model to /data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/효성/20190308152428_weights.002-0.9871.hdf5
Epoch 3/13
20907/20907 [==============================] - 16s 781us/step - loss: 0.0214 - acc: 0.9922 - val_loss: 0.0238 - val_acc: 0.9905

Epoch 00003: val_loss improved from 0.03529 to 0.02382, saving model to /data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/효성/2019030

## Predict

In [58]:
loaded_model = tf.keras.models.load_model('/data/jupyter/user/kdh/AI_Theme/KR_Homonym_Stock_Classification/checkpoint/효성/20190308152428_weights.005-0.9901.hdf5')

/data/venv/lib64/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [59]:
def predict(model, sentence):
    tokens = tokenize(sentence)
    encodes = encode(tokens)
    tf_input = np.array(encodes).reshape((1, seq_size))
    pred = model.predict(tf_input)[0][0]
    result = 1 if pred >= 0.85 else 0
    return result, pred

In [60]:
text = '코스피 중형주에 LG이노텍·한샘·동서·녹십자·효성·만도 드 33개 종목 새롭게 포함돼'
pred = predict(loaded_model, text)
pred

(1, 0.9995846)

In [61]:
text = 'HDC현대산업개발, 애경산업, 효성티앤씨, 롯데정보통신, 진에어, 한일시멘트, 효성첨단소재, 효성화학, 하나제약, 테웨이항공, 효성중공업 등은 중형주에 신규로 편입됐다.'
pred = predict(loaded_model, text)
pred

(1, 0.99999976)

In [62]:
text = """한편 이 연구소는 효성[004800]의 손병두 전 전국경제인연합회(전경련) 부회장 사외이사 재선임 안건에 대해서도 "사외이사로 재직하던 2014년 조석래 회장·조현준 사장 등 경영진의 횡령 등 불법행위에 대해 아무 조치를 하지 않고 이사로서 임무를 방기했다"며 '반대'를 권고했다"""
pred = predict(loaded_model, text)
pred

(1, 0.9999964)

In [63]:
text = '그 누구보다 당당하고 자신감 넘치는 모습, 잠재력과 성장 가능성 등을 인정받아 KLPGA에 입회하자마자 NH투자증권 프로골프단에 합류한 이가영은 2019시즌 개막전으로 열린 ‘효성 챔피언십 with SBS Golf’로 정규투어 데뷔전을 가졌다'
pred = predict(loaded_model, text)
pred

(1, 0.99998116)

In [64]:
text = '실효성이 떨어지기 때문에 기류변화가 일어나 대기정체가 풀리고 비가 내려야 그나마 미세먼지 지옥이 다소 풀릴 수 있을 것으로 보인다'
pred = predict(loaded_model, text)
pred

(0, 2.2295658e-06)

In [65]:
text = '풀러스는 7일 "국민의 이동 편익을 증가시키기 위한 당초 취지의 대타협 기구였는데, 실효성 있는 결론은 아닌 것 같다"며 "특히 시민들이 택시가 안 잡혀서 불편을 겪는 시간대에 카풀을 투입할 수 없게 되어 유감이고, 시민들이 공감할 수 없는 결론"이라고 설명했다'
pred = predict(loaded_model, text)
pred

(0, 1.4534938e-11)

In [66]:
text = '외부인 출입이 엄격히 제한된 가운데 이 전 대통령 자택 앞은 평소와 다를 바 없이 한산했지만, 보석 실효성 논란은 여전합니다'
pred = predict(loaded_model, text)
pred

(0, 1.41822665e-11)

In [67]:
text = '하지만 대포폰을 사용하거나, 가사 도우미 등 외부접촉을 원천 차단할 수는 없어 실효성이 낮다는 지적이 나옵니다'
pred = predict(loaded_model, text)
pred

(0, 8.071381e-13)

In [68]:
text = '한편 대전 한특위는 ▲중앙 한특위 정책의 지역내 홍보 ▲지역내 한방 불법 행위 감시 및 제보 접수 ▲지자체 한방 지원 사업의 유효성 검증 및 대응책 마련 ▲대전 지역 국회의원들에게의 정책 제안 등의 역할을 수행할 예정이다'
pred = predict(loaded_model, text)
pred

(0, 2.8244456e-09)

In [69]:
text = '한방 불법행위 감시·지자체 한방 지원 유효성 등 검증'
pred = predict(loaded_model, text)
pred

(0, 2.0630415e-09)

In [70]:
text = '한약 표준화 사업·한약유효성검사센터 건립 ‘집중 추진’'
pred = predict(loaded_model, text)
pred

(0, 1.591822e-09)

In [71]:
text = '특히 김 지사는 화순에 이어 방문한 장흥군청에서 가진 ‘장흥군민과 대화’에서 “한약 표준화사업, 한약 유효성검사센터 건립 사업과 펩타이드 원료시설 구축을 집중적으로 추진할 계획”이라며 생물의약산업벨트의 성공적 구축을 재차 강조했다'
pred = predict(loaded_model, text)
pred

(0, 9.140877e-09)

In [72]:
text = '김영록 전라남도지사는 7일 장흥군청에서 연 ‘장흥군민과 대화’에서 “펩타이드 원료시설 구축과 한약 표준화사업, 한약 유효성검사센터 건립 사업을 집중적으로 추진할 계획”이라며 생물의약산업벨트의 성공적 구축을 재차 강조했다'
pred = predict(loaded_model, text)
pred

(0, 1.0029414e-08)

In [73]:
text = '효성산 동네체육시설, 효성약수터 등 주변 환경정화 활동을 실시'
pred = predict(loaded_model, text)
pred

(0, 0.23499012)

In [74]:
text = '인천 계양구 효성1동 주민자치위원회(위원장 최복수) 사회진흥분과위원회는 지난 19일 효성산 동네체육시설, 효성약수터 등 주변 환경정화 활동을 실시하였다'
pred = predict(loaded_model, text)
pred

(0, 0.17965186)

In [75]:
text = '행정복지센터에서 효성산 입구까지는 무분별하게 버리고 간 쓰레기가 많아 눈살을 찌푸렸으나, 막상 효성산에 올라 환경정화 활동 시에는 쓰레기가 많지 않아 내 고장에 대한 애향심 강화 및 환경보호에 대한 주민들의 의식이 많이 변화되었음을 느껴 참여자 모두 뿌듯함을 느꼈다고 한다'
pred = predict(loaded_model, text)
pred

(0, 0.03252442)

In [76]:
text = '배효순 사회진흥분과위원장은 “우리가 살고 있는 지역의 자연을 보호하는 활동에 주민자치위원들이 솔선수범하는 뜻깊은 시간이 되었으며, 매월 실시하는 효성산 환경정화 활동에 많은 주민들이 함께 할 수 있도록 다양한 방법을 강구하여 우리 삶의 터전인 산을 보호해야 한다.”라고 말했다'
pred = predict(loaded_model, text)
pred

(0, 0.018406373)

In [77]:
text = '전효성이 화이트데이를 맞이해 팬미팅을 개최한다. 팬들과 특별한 시간을 보낸다'
pred = predict(loaded_model, text)
pred

(0, 6.4637126e-05)

In [78]:
text = "0일 전효성이 합정 라디오가가극장에서 팬들에게 받은 사랑을 보답하고자 '화이트데이 앤 효성'을 개최해 1여년 만에 팬들과 만난다"
pred = predict(loaded_model, text)
pred

(0, 0.11132557)

In [79]:
text = "한국여자프로골프협회(KLPGA) 2019시즌의 포문을 열 ‘효성 챔피언십 with SBS Golf’가 오는 12월 7일(금)부터 사흘간 베트남에 위치한 트윈도브스 골프클럽(파72/6,579야드)에서 막을 올렸다"
pred = predict(loaded_model, text)
pred

(1, 0.9999981)

In [80]:
text = '효성그룹 소속 섬유·무역업체 효성티앤씨가 대구에서 고객사들과 판로 확장에 몰두하고 있다'
pred = predict(loaded_model, text)
pred

(1, 1.0)

In [81]:
text = '7일 업계에 따르면 효성티앤씨는 지난 6일부터 미광, 보광, 삼성교역 등 11개 고객사와 2019 대구국제섬유박람회에 참가 중이다'
pred = predict(loaded_model, text)
pred

(1, 0.9999745)

In [82]:
text = '효성그룹은 2002년 1회부터 지금까지 한 해도 거르지 않고 자신들의 섬유 제품을 소개하면서 고객사 활동도 뒷받침하고 있다'
pred = predict(loaded_model, text)
pred

(1, 0.99999917)

In [83]:
text = '이번 박람회에서 효성티앤씨는 일반 스판덱스보다 저온에서 제작돼 촉감이 부드러운 크레오라 에코소프트, 색깔이 뚜렷하고 착용감도 좋은 크레오라 컬러플러스를 선보였다'
pred = predict(loaded_model, text)
pred

(1, 0.99997616)

In [84]:
text = '아울러 효성티앤씨는 △자외선 차단 기능을 가진 냉감성 섬유 마이판 아쿠아-엑스 △빛에너지를 열에너지로 바꾸는 섬유 에어로히트 △페트병을 재활용한 폴리에스터 리젠 △효성 독자 기술로 개발한 탄소섬유 탄섬 △고열에 강한 아라미드 원사 알켁스 등을 전시했다'
pred = predict(loaded_model, text)
pred

(1, 0.9999999)

In [85]:
text = '김용섭 효성티앤씨 대표는 "올해 목표인 고객 만족 극대화를 이루기 위해 소비자와 자주 만나려 한다"며 "차별화된 제품으로 섬유시장을 이끌겠다"고 밝혔다'
pred = predict(loaded_model, text)
pred

(1, 0.9999957)

In [86]:
text = '효성첨단소재가 전주 탄소섬유 공장 증설을 위해 전북도와 8일 투자 협약을 맺었다'
pred = predict(loaded_model, text)
pred

(1, 1.0)

In [87]:
text = '효성첨단소재는 내년 2월까지 전주 친환경첨단복합단지 내 탄소섬유 생산공장 인근 18만㎡ 부지에 468억원을 투자해 생산라인을 추가로 만든다'
pred = predict(loaded_model, text)
pred

(1, 0.9999999)

In [88]:
text = '효성 관계자는 "미래 친환경 자동차로 주목 받는 수소차 및 CNG 차량, 그리고 전선심재 경량화 핵심 소재이자 산업∙항공용 첨단소재인 탄소섬유의 수요 증가에 대비하기 위한 것"이라고 설명했다'
pred = predict(loaded_model, text)
pred

(1, 1.0)

In [89]:
text = '효성은 지난 2015년부터 시내버스 CNG 고압용기용 탄소섬유를 납품해 왔다'
pred = predict(loaded_model, text)
pred

(1, 0.99999034)

In [90]:
text = "효성은 지난 2007년 탄소섬유 개발에 뛰어든 이후 4년만인 2011년 국내 최초 독자 개발한 고성능 탄소섬유(브랜드명 탄섬 'TANSOME')을 상업화했다"
pred = predict(loaded_model, text)
pred

(1, 1.0)

In [91]:
text = '효성그룹 지주사 효성이 좋은 현금흐름을 바탕으로 2019년에도 높은 배당금을 유지할 것으로 전망됐다'
pred = predict(loaded_model, text)
pred

(1, 1.0)

In [92]:
text = '이상헌 한국투자증권 연구원은 6일 “효성의 배당금 책정은 앞으로 현금흐름이 어떻게 되느냐가 관건”이라며 “효성은 배당여력이 증가할 여지가 많아 높은 배당금을 유지할 수 있을 것”이라고 내다봤다'
pred = predict(loaded_model, text)
pred

(1, 0.9999212)

In [93]:
text = '효성은 지난 2월 2018년 결산배당을 1주당 5천 원으로 결정했다'
pred = predict(loaded_model, text)
pred

(1, 0.9996371)